In [1]:
import pandas as pd

In [10]:
df = pd.read_csv('africa_merged_dataset.csv')
solar = pd.read_csv('../../energy_strength/solar/solar_countries.csv')
solar = solar[solar['iso_a3'].isin(list(df['iso3_code']))]
wind = pd.read_csv('../../energy_strength/wind/wind_countries.csv')
wind = wind[wind['iso_a3'].isin(list(df['iso3_code']))]


In [11]:
df.sample(5)

,lat,long,population,wind_speed,solar,iso2_code,name,iso3_code,corruption_perception_index,conflict_intensity
120546,19.333333,-12.375000,1.960801,7.193729,NaN,MR,Mauritania,MRT,28,95
534863,-22.000000,34.541667,238.999800,5.955882,NaN,MZ,Mozambique,MOZ,26,777
352228,0.458333,12.625000,52.801260,1.826559,219.564026,GA,Gabon,GAB,31,98
534336,-21.291667,34.791667,250.745200,6.389903,NaN,MZ,Mozambique,MOZ,26,777
215879,5.958333,-3.916667,1836.404000,4.432179,NaN,CI,Côte d'Ivoire,CIV,36,792


In [36]:
solar.sample(5)

,Unnamed: 0,lat,lon,ALLSKY_SFC_SW_DWN,name,iso_a3
40490,40490,22.5,-9.5,261.027344,Mauritania,MRT
30817,30817,-4.5,37.5,224.535156,Tanzania,TZA
29358,29358,-8.5,18.5,214.363281,Angola,AGO
32232,32232,-0.5,12.5,180.742188,Gabon,GAB
32966,32966,1.5,26.5,200.562500,Dem. Rep. Congo,COD


In [28]:
wind.sample(5)

,Unnamed: 0,lat,lon,WS50M,name,iso_a3
108628,108628,4.0,32.500,3.230469,S. Sudan,SSD
136861,136829,28.5,18.125,5.855469,Libya,LBY
85573,85573,-16.0,23.125,4.750000,Zambia,ZMB
113748,113748,8.5,-7.500,3.945312,Côte d'Ivoire,CIV
81560,81560,-19.5,35.000,4.917969,Mozambique,MOZ
